In [51]:
from twelvedata import TDClient
import pprint
import pandas as pd
import time
import sqlite3
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
td = TDClient(apikey="faa93033c87f4b8290b9486920e97a86")

In [52]:
# Cryptocurrencies List, td.get_cryptocurrencies_list() see at: (https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python)
all_Cryptos = td.get_cryptocurrencies_list().as_json()

#Print firtst item in list as visual aid
all_Cryptos[0]

{'symbol': '0xBTC/BTC',
 'available_exchanges': ['Hotbit', 'Mercatox'],
 'currency_base': '0xBitcoin',
 'currency_quote': 'Bitcoin'}

In [53]:
# Filter the symbols that are being compared to USD
usd_symbols = [crypto['symbol'] for crypto in all_Cryptos if '/USD' in crypto['symbol']]
print("Number of USD-paired symbols:", len(usd_symbols))

Number of USD-paired symbols: 1641


In [54]:
# Time Series example
ts = td.time_series(
    symbol="BTC/USD",
    timezone="America/New_York",
    interval="30min",
    outputsize=5
).as_pandas()
ts

,open,high,low,close
datetime,,,,
2023-07-24 21:00:00,29113.94,29116.60,29062.33,29070.07
2023-07-24 20:30:00,29174.04,29181.16,29092.06,29113.95
2023-07-24 20:00:00,29176.98,29184.59,29149.08,29174.24
2023-07-24 19:30:00,29185.18,29191.62,29168.57,29176.98
2023-07-24 19:00:00,29160.04,29206.97,29154.14,29185.33


In [55]:
# Create a list to store the DataFrames for each symbol in the batch
crypto_list = []

# Loop through batches of symbols
symbol_batch = ['BTC/USD', 'ETH/USD', 'USDT/USD', 'XRP/USD', 
                'BNB/USD', 'ADA/USD', 'SOL/USD', 'DOGE/USD']

# Process each symbol in the current batch
for symbol in symbol_batch:
    try:
        ts = td.time_series(
            symbol=symbol,
            interval="1week",
            timezone="America/New_York",
            outputsize=208
        ).with_bbands(time_period=3).with_macd(fast_period=4, slow_period=8, signal_period=2).with_rsi(time_period=6).as_pandas()

        # Rename the 'close' column to the symbol name for clarity
        #ts.rename(columns={'close': symbol}, inplace=True)

        # Add a new numeric index column without dropping the original timestamp index
        ts.reset_index(inplace=True)

        # Add the DataFrame for the current symbol to the list
        crypto_list.append(ts)

    except Exception as e:
        print(f"Error fetching data for symbol '{symbol}': {str(e)}")

# Introduce a delay of 1 minute between API calls
time.sleep(60)

print("Total DataFrames fetched:", len(crypto_list))

Total DataFrames fetched: 8


In [56]:
crypto_list[1]

,datetime,open,high,low,close,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,rsi
0,2023-07-24,1888.81000,1890.94000,1833.52000,1846.02000,1948.24282,1885.73667,1823.23051,4.60473,8.65686,-4.05213,48.07676
1,2023-07-17,1921.95000,1935.97000,1846.84000,1888.80000,1940.08518,1891.35333,1842.62149,15.90139,16.76113,-0.85974,54.00389
2,2023-07-10,1862.64000,2029.66000,1846.52000,1922.39000,1972.32294,1907.69333,1843.06373,20.77873,18.48060,2.29813,58.74129
3,2023-07-03,1937.83000,1977.16000,1827.00000,1862.87000,1960.98186,1899.76333,1838.54480,14.47613,13.88433,0.59179,52.60154
4,2023-06-26,1898.98000,1959.99000,1815.84000,1937.82000,2041.19584,1852.44333,1663.69083,20.88747,12.70075,8.18673,62.33571
...,...,...,...,...,...,...,...,...,...,...,...,...
93,2021-10-11,3413.80005,3966.47998,3377.53003,3847.54004,3966.50171,3560.35335,3154.20500,0.00000,0.00000,0.00000,0.00000
94,2021-10-04,3416.73999,3667.88989,3274.86011,3415.52002,3631.98430,3299.05330,2966.12231,0.00000,0.00000,0.00000,0.00000
95,2021-09-27,3058.71997,3484.60010,2783.83008,3418.00000,3571.20103,3270.18327,2969.16550,0.00000,0.00000,0.00000,0.00000
96,2021-09-20,3328.98999,3346.88989,2653.58008,3063.63989,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [57]:
# Create a database connection using SQLAlchemy
engine = create_engine('sqlite:///crypto_data.db')

Base = automap_base()
Base.prepare(autoload_with=engine)

# Loop through the DataFrames in 'crypto_list' and the symbols in 'symbol_batch'
for df, symbol in zip(crypto_list, symbol_batch):
    table_name = f'crypto_{symbol.replace("/", "_")}'  # Create a unique table name based on the symbol
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
# Close the database connection
#engine.dispose()

In [62]:
# Replace 'table_name' with the specific table you want to show
table_name = 'crypto_BTC_USD'

df = pd.read_sql_table(table_name, engine)
# Convert the DataFrame to JSON format
json_data = df.to_json(orient='records')
pprint.pprint(json_data)

'[{"datetime":1690156800000,"open":30081.61,"high":30099.99,"low":28850.0,"close":29079.71,"upper_band":30834.79846,"middle_band":29802.98667,"lower_band":28771.17487,"macd":327.86126,"macd_signal":437.0138,"macd_hist":-109.15254,"rsi":51.73036},{"datetime":1689552000000,"open":30247.56,"high":30421.29,"low":29525.0,"close":30081.61,"upper_band":30301.48581,"middle_band":30165.87667,"lower_band":30030.26752,"macd":616.69655,"macd_signal":655.31888,"macd_hist":-38.62233,"rsi":63.75839},{"datetime":1688947200000,"open":30168.26,"high":31862.21,"low":29920.31,"close":30247.64,"upper_band":30730.92442,"middle_band":30343.17667,"lower_band":29955.42891,"macd":736.48711,"macd_signal":732.56355,"macd_hist":3.92357,"rsi":65.87354},{"datetime":1688342400000,"open":30613.57,"high":31525.1,"low":29715.87,"close":30168.38,"upper_band":30793.78031,"middle_band":30420.71,"lower_band":30047.63969,"macd":789.6484,"macd_signal":724.71641,"macd_hist":64.93199,"rsi":65.41713},{"datetime":1687737600000,"o